In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
from datetime import datetime, timedelta
import json

import glob
import json
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource, save
from iot.core import get_bitalino_df, get_glasses_df, get_tablet_df, build_graphs



In [3]:
output_path = 'C:/Users/artem/PycharmProjects/iot/output'
sync_names = ['glasses', 'tablet']
base_name  = 'bitalino'
experiment_file = 'experiments/exp3.txt'
experiment_name = experiment_file.split('/')[-1].split('.')[0]

In [4]:
with open(experiment_file) as f:
    lines = f.read().split('\n')[:3]

In [5]:
bitalino_path, glasses_path, tablet_path = lines

In [6]:
dic_df = {}
if bitalino_path:
    dic_df['bitalino'], _ = get_bitalino_df(bitalino_path, N = 100000)
dic_df['glasses'], _ = get_glasses_df(glasses_path, N = 10000)
if tablet_path:
    dic_df['tablet'], _ = get_tablet_df(tablet_path, N = 10000)

In [8]:
build_graphs(dic_df, base_name, sync_names, experiment_name, output_path)